In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, re

pd.set_option('display.max_columns', 80)
pd.set_option('display.max_rows', 400)

### Step 1) Making Data Description Table as a DataFrame (Using pandas)
- 1-1. Read a text file
- 1-2. Use Regular Expression for Parsing
- 1-3. Input the data into DataFrame

In [2]:
data_type_df = pd.DataFrame(columns=['Data_Type', 'Column', 'Column_Detail', 'Value', 'Value_Detail'])

In [3]:
a = open('./data_description.txt')
index = 0
for i in a.readlines():
    col_p = re.match('[\w]+:[\$\w\s]+', i.strip())
    val_p = re.match('[\s]+[\w]+[\t\W\-\(\)\/\&\d\:\+\<\s\w]+', i)
    if col_p:
        column = col_p.group().split(': ')
        data_type_df.loc[index, ('Data_Type', 'Column', 'Column_Detail')] = ('Numerical', column[0], column[1])
        index += 1
    elif val_p and col_p == None:
        value = val_p.group().strip().split('\t')
        data_type_df.loc[(index-1), ('Data_Type', 'Column', 'Value', 'Value_Detail')] = ('Categorical', column[0], value[0], value[1])
        index += 1

In [4]:
data_type_df

,Data_Type,Column,Column_Detail,Value,Value_Detail
0,Categorical,MSSubClass,Identifies the type of dwelling involved in th...,20,1-STORY 1946 & NEWER ALL STYLES
1,Categorical,MSSubClass,NaN,30,1-STORY 1945 & OLDER
2,Categorical,MSSubClass,NaN,40,1-STORY W/FINISHED ATTIC ALL AGES
3,Categorical,MSSubClass,NaN,45,1-1/2 STORY - UNFINISHED ALL AGES
4,Categorical,MSSubClass,NaN,50,1-1/2 STORY FINISHED ALL AGES
5,Categorical,MSSubClass,NaN,60,2-STORY 1946 & NEWER
6,Categorical,MSSubClass,NaN,70,2-STORY 1945 & OLDER
7,Categorical,MSSubClass,NaN,75,2-1/2 STORY ALL AGES
8,Categorical,MSSubClass,NaN,80,SPLIT OR MULTI-LEVEL
9,Categorical,MSSubClass,NaN,85,SPLIT FOYER


### Step 2) Export DataFrame to Database
- 2-1. MySQL
- 2-2. MongoDB

#### 2-1. MySQL

In [5]:
import pymysql.cursors
from sqlalchemy import create_engine

In [6]:
password = 'password'
db_name = 'kaggle_house_price'
engine = create_engine("mysql://root:"+"%s" % password+"@localhost/%s" % db_name, encoding='utf-8')

In [7]:
conn = engine.connect()

###### Export to MySQL

In [8]:
table_name = 'data_desc'
data_type_df.to_sql(name=table_name, con=conn, if_exists='replace')

###### Import from MySQL

In [9]:
select_sql = "SELECT * FROM %s" % table_name
df = pd.read_sql(sql=select_sql, con=conn)

In [10]:
df.head()

,index,Data_Type,Column,Column_Detail,Value,Value_Detail
0,0,Categorical,MSSubClass,Identifies the type of dwelling involved in th...,20,1-STORY 1946 & NEWER ALL STYLES
1,1,Categorical,MSSubClass,None,30,1-STORY 1945 & OLDER
2,2,Categorical,MSSubClass,None,40,1-STORY W/FINISHED ATTIC ALL AGES
3,3,Categorical,MSSubClass,None,45,1-1/2 STORY - UNFINISHED ALL AGES
4,4,Categorical,MSSubClass,None,50,1-1/2 STORY FINISHED ALL AGES


In [11]:
df.shape

(353, 6)

In [12]:
conn.close()